In [41]:
!pip install ipython-autotime
%load_ext autotime
import pandas as pd
from google.colab import drive
import pathlib
import numpy as np
import gc
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dropout,Dense
from keras.callbacks import ModelCheckpoint
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


drive.mount('/content/drive',force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Mounted at /content/drive
time: 6.01 s (started: 2022-11-26 04:29:13 +00:00)


In [2]:
data_dir = "/content/drive/MyDrive/yellow_tripdata_2020_data.parquet"
data_dir = pathlib.Path(data_dir)
yellowTaxiData2020 = pq.ParquetFile(data_dir)
del data_dir

time: 21.3 ms (started: 2022-11-26 02:21:48 +00:00)


In [3]:
random_state=17

time: 503 µs (started: 2022-11-26 02:21:48 +00:00)


In [4]:
taxiData = pd.DataFrame()

time: 1.38 ms (started: 2022-11-26 02:21:48 +00:00)


In [5]:
for taxiDataBatch in yellowTaxiData2020.iter_batches(batch_size=2000000):
    taxiDataBatchDF = taxiDataBatch.to_pandas()
    taxiDataBatchDF.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_date', 'pickup_time', 'dropoff_date', 'dropoff_time'], inplace=True)
    taxiData = pd.concat([taxiData, taxiDataBatchDF])

time: 2.82 s (started: 2022-11-26 02:21:48 +00:00)


In [6]:
del yellowTaxiData2020
gc.collect()

44

time: 172 ms (started: 2022-11-26 02:21:51 +00:00)


In [7]:
data_dir = "/content/drive/MyDrive/yellow_tripdata_2021_data.parquet"
data_dir = pathlib.Path(data_dir)
yellowTaxiData2021 = pq.ParquetFile(data_dir)
del data_dir

time: 6.09 ms (started: 2022-11-26 02:21:51 +00:00)


In [8]:
for taxiDataBatch in yellowTaxiData2021.iter_batches(batch_size=2000000):
    taxiDataBatchDF = taxiDataBatch.to_pandas()
    taxiDataBatchDF.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_date', 'pickup_time', 'dropoff_date', 'dropoff_time'], inplace=True)
    taxiData = pd.concat([taxiData, taxiDataBatchDF])

time: 14.4 s (started: 2022-11-26 02:21:51 +00:00)


In [9]:
def createLSTMDataSet(data, sequence_length, feature_count): 
    X = []
    y = []

    for i in range(data.shape[0] - feature_count - 1):
        if data[i, 0] == data[i+1, 0]:
            if data[i, 0] == data[i+sequence_length, 0]:
                X.append(data[i:i+sequence_length, :feature_count])
                y.append(data[i+sequence_length, feature_count])
    
    return np.array(X), np.array(y)


time: 1.58 ms (started: 2022-11-26 02:22:06 +00:00)


In [11]:
taxiDataLSTM = taxiData[['PULocationID', 'DOLocationID', 'pickup_weekday', 'pickup_month', 'pickup_dt', 'trip_distance', 'pickup_holiday', 'pickup_near_holiday', 'ride_duration']]

time: 7.94 s (started: 2022-11-26 02:22:06 +00:00)


In [12]:
taxiDataLSTM.sort_values(by=['PULocationID', 'DOLocationID', 'pickup_weekday', 'pickup_month', 'pickup_dt'], inplace=True, ascending=True)

time: 36.6 s (started: 2022-11-26 02:22:14 +00:00)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [13]:
del taxiData
gc.collect()

304

time: 177 ms (started: 2022-11-26 02:22:51 +00:00)


In [14]:
scaler = MinMaxScaler()
taxiDataLSTM = scaler.fit_transform(taxiDataLSTM)

time: 2.42 s (started: 2022-11-26 02:22:51 +00:00)


In [15]:
X, y = createLSTMDataSet(taxiDataLSTM, 5, 8)

time: 1min 14s (started: 2022-11-26 02:22:59 +00:00)


In [16]:
del yellowTaxiData2021, taxiDataBatchDF
gc.collect()

131

time: 165 ms (started: 2022-11-26 02:24:20 +00:00)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.4, 
                                                    random_state=random_state)

time: 8.7 s (started: 2022-11-26 02:24:20 +00:00)


In [18]:
np.random.seed(random_state)

time: 680 µs (started: 2022-11-26 02:28:02 +00:00)


In [21]:
del X, y
gc.collect()

345

time: 193 ms (started: 2022-11-26 02:29:16 +00:00)


In [31]:
model = Sequential()
model.add(LSTM(units=128, input_shape=(5,8), return_sequences=True))
model.add(Dropout(0.2, seed=random_state))
model.add(LSTM(units=128, return_sequences=False))
model.add(Dropout(0.2, seed=random_state))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')

time: 725 ms (started: 2022-11-26 02:51:31 +00:00)


In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 5, 128)            70144     
                                                                 
 dropout_2 (Dropout)         (None, 5, 128)            0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 201,857
Trainable params: 201,857
Non-trainable params: 0
_________________________________________________________________
time: 35.9 ms (started: 2022-11-26 02:51:32 +00:

In [33]:
gc.collect()

7199

time: 237 ms (started: 2022-11-26 02:51:34 +00:00)


In [34]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.01, mode='min')

time: 787 µs (started: 2022-11-26 02:51:36 +00:00)


In [35]:
model.fit(X_train, y_train, epochs=5, batch_size=714,shuffle=True, callbacks=[callback])

Epoch 1/5
21492/21492 [==============================] - 1569s 73ms/step - loss: 0.0384
Epoch 2/5
21492/21492 [==============================] - 1559s 73ms/step - loss: 0.0362


time: 52min 9s (started: 2022-11-26 02:51:50 +00:00)


In [36]:
model.save('/content/drive/MyDrive/Colab Notebooks/lstm_ny_model')

time: 10.2 s (started: 2022-11-26 03:44:43 +00:00)


In [37]:
predictions = model.predict(X_test)

319693/319693 [==============================] - 2462s 8ms/step
time: 42min 52s (started: 2022-11-26 03:45:17 +00:00)


In [38]:
predictions

array([[0.5366985 ],
       [0.41321748],
       [0.37551996],
       ...,
       [0.31194037],
       [0.4220124 ],
       [0.38538814]], dtype=float32)

time: 4.06 ms (started: 2022-11-26 04:28:12 +00:00)


In [39]:
y_test

array([0.26746725, 0.28657205, 0.54421397, ..., 0.35917031, 0.16757642,
       0.4121179 ])

time: 3.67 ms (started: 2022-11-26 04:28:20 +00:00)


In [43]:
predictions_train = model.predict(X_train)

479539/479539 [==============================] - 4308s 9ms/step
time: 1h 14min 33s (started: 2022-11-26 04:29:53 +00:00)


In [44]:
rmse_train = mean_squared_error(y_train, predictions_train)**0.5
print(f'The RMSE of training prediction is: {rmse_train}')

The RMSE of training prediction is: 0.1877786177459142
time: 272 ms (started: 2022-11-26 05:45:41 +00:00)


In [45]:
mse_train = mean_squared_error(y_train, predictions_train)
print(f'The MSE of training prediction is: {mse_train}')

The MSE of training prediction is: 0.03526080928256616
time: 100 ms (started: 2022-11-26 05:45:43 +00:00)


In [46]:
mae_train = mean_absolute_error(y_train, predictions_train)
print(f'The MAE of training prediction is: {mae_train}')

The MAE of training prediction is: 0.15122186322047848
time: 358 ms (started: 2022-11-26 05:45:47 +00:00)


In [42]:
rmse_test = mean_squared_error(y_test, predictions)**0.5
print(f'The RMSE of prediction is: {rmse_test}')

The RMSE of prediction is: 0.18786428847575676
time: 280 ms (started: 2022-11-26 04:29:25 +00:00)


In [47]:
mse_test = mean_squared_error(y_test, predictions)
print(f'The MSE of prediction is: {mse_test}')

The MSE of prediction is: 0.03529299088450235
time: 77.3 ms (started: 2022-11-26 05:45:52 +00:00)


In [48]:
mae_test = mean_absolute_error(y_test, predictions)
print(f'The MAE of training prediction is: {mae_test}')

The MAE of training prediction is: 0.1513102728706388
time: 186 ms (started: 2022-11-26 05:45:57 +00:00)
